In [1]:
import torch
from torch import nn
from torch.nn import functional as F

#自定义不带参数的层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,X):
        return X-X.mean()

In [3]:
layer=CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5,6]))

tensor([-2.5000, -1.5000, -0.5000,  0.5000,  1.5000,  2.5000])

In [4]:
net=nn.Sequential(nn.Linear(8,128),layer)

In [6]:
Y=net(torch.rand(4,8))
Y.mean()

tensor(-1.3970e-09, grad_fn=<MeanBackward0>)

In [16]:
#自定义带参数的层
class MyLinear(nn.Module):
    def __init__(self,in_units,out_units):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_units,out_units))
        self.bias=nn.Parameter(torch.randn(out_units,))  #在out_unit后面加个逗号是为了表明他是一个单元素的元组而不是一个数
    
    def forward(self,X):
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)

In [17]:
linear=MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[ 0.8953, -3.1012,  1.4052],
        [ 0.5106, -0.4202, -1.2366],
        [ 0.9890,  0.7775,  2.2102],
        [-0.1822, -0.5898,  0.4429],
        [ 0.7657,  0.5951,  0.0441]], requires_grad=True)

In [18]:
linear(torch.randn(2,5))

tensor([[1.5211, 0.0140, 3.6944],
        [0.0000, 0.2146, 0.0000]])

In [19]:
net=nn.Sequential(MyLinear(4,64),MyLinear(64,8))  #使用自定义的层定义网络，只要这个层继承了nn.Module就可放到Sequential中去
net(torch.randn(2,4))

tensor([[ 0.0000,  0.0000,  0.0000,  0.8213,  0.9081,  5.3487, 19.3107,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  5.8315,  2.0861,  6.8296, 13.4528,  0.0000]])

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x=torch.arange(4)
torch.save(x,'x-file')  #保存数据为文件

In [2]:
x2=torch.load('x-file') #读取保存的文件
print(x2)

In [8]:
#存储一个张量列表，然后把它们读回内存。
x,y=torch.tensor([[1,4,5],[6,8,9]]),torch.randn((2,4))
torch.save([x,y],'x-file')
x2,y2=torch.load('x-file')
x2,y2

(tensor([[1, 4, 5],
         [6, 8, 9]]),
 tensor([[ 1.3356, -0.7694,  0.3573, -1.2055],
         [-0.0643,  0.1018,  0.3931,  1.2878]]))

In [9]:
#写入或读取从字符串映射到张量的字典
mydict={'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2=torch.load('mydict')
mydict2

{'x': tensor([[1, 4, 5],
         [6, 8, 9]]),
 'y': tensor([[ 1.3356, -0.7694,  0.3573, -1.2055],
         [-0.0643,  0.1018,  0.3931,  1.2878]])}

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1=nn.Linear(20,256)
        self.output=nn.Linear(256,10)
    def forward(self,X):
        return self.output(F.relu(self.hidden1(X)))

net=MLP()
X=torch.randn(size=(2,20))
net(X)

tensor([[-0.6803,  0.2999, -0.0912, -0.4366, -0.3621, -0.0164, -0.3231, -0.0087,
          0.0446, -0.2850],
        [ 0.2597,  0.0372,  0.3711, -0.0743, -0.1652,  0.0736, -0.2700, -0.2010,
          0.0298, -0.2397]], grad_fn=<AddmmBackward0>)

In [14]:
torch.save(net.state_dict(),'mlp.params')

In [17]:
clone=MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden1): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [28]:
Y=net(X)
Y_clone=clone(X)
Y_clone==Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])